In [ ]:
import numpy as np
#import scipy
from scipy import integrate, LowLevelCallable
from scipy.integrate import odeint,solve_ivp
import matplotlib.pyplot as plt
import os, ctypes
import timeit

In [ ]:
def G(u, v, f, k):
    return f * (1 - u) - u*v**2

def H(u, v, f, k):
    return -(f + k) * v + u*v**2

In [ ]:
def grayscott1d(y, t, f, k, Du, Dv, dx):
    """
    Differential equations for the 1-D Gray-Scott equations.

    The ODEs are derived using the method of lines.
    """
    # The vectors u and v are interleaved in y.  We define
    # views of u and v by slicing y.
    u = y[::2]
    v = y[1::2]

    # dydt is the return value of this function.
    dydt = np.empty_like(y)

    # Just like u and v are views of the interleaved vectors
    # in y, dudt and dvdt are views of the interleaved output
    # vectors in dydt.
    dudt = dydt[::2]
    dvdt = dydt[1::2]

    # Compute du/dt and dv/dt.  The end points and the interior points
    # are handled separately.
    dudt[0]    = G(u[0],    v[0],    f, k) + Du * (-2.0*u[0] + 2.0*u[1]) / dx**2
    dudt[1:-1] = G(u[1:-1], v[1:-1], f, k) + Du * np.diff(u,2) / dx**2
    dudt[-1]   = G(u[-1],   v[-1],   f, k) + Du * (- 2.0*u[-1] + 2.0*u[-2]) / dx**2
    dvdt[0]    = H(u[0],    v[0],    f, k) + Dv * (-2.0*v[0] + 2.0*v[1]) / dx**2
    dvdt[1:-1] = H(u[1:-1], v[1:-1], f, k) + Dv * np.diff(v,2) / dx**2
    dvdt[-1]   = H(u[-1],   v[-1],   f, k) + Dv * (-2.0*v[-1] + 2.0*v[-2]) / dx**2

    return dydt

In [ ]:
# def f(y,t):
#     dydt = y*np.exp(-t)
#     return dydt
# t = np.linspace(0,10) # linspace(start,stop,num =?)
# y0 = 0.2
# y = odeint(f,y0,t)

# plt.figure(figsize = (6,4))
# plt.plot(t,y)
# plt.xlabel('Time (sec)')
# plt.ylabel('y(t)')
# plt.show()


In [ ]:
rng = np.random.default_rng()

y0 = rng.standard_normal(5000)

t = np.linspace(0, 50, 100)

In [ ]:
f = 0.024
k = 0.055
Du = 0.01
Dv = 0.005
dx = 0.025

In [ ]:
#Time the computation without taking advantage of the banded structure of the Jacobian matrix:


In [ ]:
#%timeit sola = odeint(grayscott1d, y0, t, args=(f, k, Du, Dv, dx))
#%timeit sola = odeint(grayscott1d, y0, t, args=(f, k, Du, Dv, dx))

In [ ]:
#Now set ml=2 and mu=2, so odeint knows that the Jacobian matrix is banded:

In [ ]:
#%timeit solb = odeint(grayscott1d, y0, t, args=(f, k, Du, Dv, dx), ml=2, mu=2)
#solb = odeint(grayscott1d, y0, t, args=(f, k, Du, Dv, dx), ml=2, mu=2)


In [ ]:
#That is quite a bit faster!
#Let’s ensure that they have computed the same result:

In [ ]:
#np.allclose(sola, solb)
#solb

In [ ]:
# ode_fn = lambda t, x: np.sin(t) + 3. * np.cos(2. * t) - x

# an_sol = lambda t : (1./2.) * np.sin(t) - (1./2.) * np.cos(t) + \
#                     (3./5.) * np.cos(2.*t) + (6./5.) * np.sin(2.*t) - \
#                     (1./10.) * np.exp(-t)
# t_begin=0.
# t_end=10.
# t_nsamples=100
# t_space = np.linspace(t_begin, t_end, t_nsamples)
# x_init = 0.

# x_an_sol = an_sol(t_space)

# method = 'RK45' #available methods: 'RK45', 'RK23', 'DOP853', 'Radau', 'BDF', 'LSODA'
# num_sol = solve_ivp(ode_fn, [t_begin, t_end], [x_init], method=method, dense_output=True)
# x_num_sol = num_sol.sol(t_space).T

# plt.figure()
# plt.plot(t_space, x_an_sol, '--', linewidth=2, label='analytical')
# plt.plot(t_space, x_num_sol, linewidth=1, label='numerical')
# plt.title('ODE 1st order IVP solved by SciPy with method=' + method)
# plt.xlabel('t')
# plt.ylabel('x')
# plt.legend()
# plt.show()

In [ ]:
# # Reaction  A+B = C and B+C = D

# def mult_rxn(c,t):
#     cA = c[0]
#     cB = c[1]
#     cC = c[2]
#     cD = c[3]
#     k1 = 1.0
#     k2 = 1.5
#     r1 = k1*cA*cB
#     r2 = k2*cB*cC
#     dAdt = -r1
#     dBdt = -r1-r2
#     dCdt = r1-r2
#     dDdt = r2
#     return [dAdt,dBdt,dCdt,dDdt]

# # for the solver
# c0 = [1,1,0,0] # initial A and B = 1; C and D = 0
# t = np.arange(0,3.01,0.2)
# c = odeint(mult_rxn,c0,t)

# cA = c[:,0]
# cB = c[:,1]
# cC = c[:,2]
# cD = c[:,3]

# # selectivity
# S = np.empty(len(cC))
# for i in range(len(cC)):
#     if abs(cC[i]+cD[i] > 1e-10):
#         S[i] = cC[i]/(cC[i]+cD[i])
#     else:
#         S[i] = 1.0
        
# plt.figure(figsize = (6,4))
# plt.plot(t,cA,label = 'cA')
# plt.plot(t,cB,label ='cB')
# plt.plot(t,cC,label = 'cC')
# plt.plot(t,cD,label ='cD')
# plt.plot(t,S,label ='Sel')
# plt.xlabel('Time (sec)')
# plt.ylabel('concentration')
# plt.legend()
# plt.savefig('mult_rxns.png')
# plt.show()

In [ ]:
# solving the Heat Equation
# given details

In [ ]:
# l = 0.1 # wall thickness
# n = 10 # num of slices
# T0 = 0 # initial wall temps
# T1s = 40 # wall surface temp. at the beginning
# T2s = 20 # wall surface temp. at the end
# dx = l/n
# alpha = 0.0001 # thermal diffisivity
# t_final = 60
# dt = 0.1

# # points to plot or visualize
# x = np.linspace(dx/2,l-dx/2,n)
# # Temp. measurements along x of the wall 
# T = np.ones(n)*T0 # 
# # derivative vector, to be over written
# dTdt = np.empty(n)
# # time vector
# t = np.arange(0,t_final,dt)

# #through time, t and space, x
# #plt.clf() didn't work
# for j in range(1,len(t)):    
#     for i in range(1,n-1):
#         dTdt[i] = alpha*(-(T[i]-T[i-1])/dx**2+(T[i+1]-T[i])/dx**2)
#     dTdt[0] = alpha*(-(T[0]-T1s)/dx**2+(T[1]-T[0])/dx**2)
#     dTdt[n-1] = alpha*(-(T[n-1]-T[n-2])/dx**2+(T2s-T[n-1])/dx**2)
#     T = T + dTdt*dt
# plt.figure()
# plt.plot(x,T)
# plt.axis([0, l,0, 50])
# plt.xlabel('Distance x, (m)')
# plt.ylabel('Temp., (C)')
# plt.show()
# plt.pause(0.01)

In [ ]:
# Solve the heat equation using the odeint solver

In [ ]:
# def heat_eqn(T,t):
#     l = 0.1
#     n = 10
#     T0 = 0
#     T1s = 40
#     T2s = 20
#     dx = l/n
#     dt = 0.1
#     alpha = 0.0001
#     t_final = 60
#     t = np.arange(0,t_final,dt)
#     T = np.ones(n)*T0
#     dTdt = np.empty(n)
    
#     for j in range(1,len(t)):
#         for i in range(1,n-1):
#             dTdt[i] = alpha*(-(T[i]-T[i-1])/dx**2+(T[i+1]-T[i])/dx**2)
#     dTdt[0] = alpha*(-(T[0]-T1s)/dx**2+(T[1]-T[0])/dx**2)
#     dTdt[n-1] = alpha*(-(T[n-1]-T[n-2])/dx**2+(T2s-T[n-1])/dx**2)
#     return dTdt

# # solve for temp. using the solver, odeint
# l = 0.1
# n = 10
# T0 = 0
# dx = l/n
# T_0 = np.ones(n)*T0 # 
# t_final = 60
# x = np.linspace(dx/2,l-dx/2,n)

# dt = 0.1
# x = np.linspace(dx/2,l-dx/2,n)
# t = np.arange(0,t_final,dt)
# T = np.ones(n)*T0
# T = T + heat_eqn*dt
# #T = odeint(heat_eqn,T_0,t) # did not work?!

# plt.figure(figsize = (6,4))
# plt.plot(x,T)
# plt.axis([0, l,0, 50])
# plt.xlabel('Distance (m)')
# plt.ylabel('Temp.(C)')
# plt.show()